In [15]:
import cv2
import os 
import numpy as np 
from sklearn.model_selection import train_test_split
from skimage.feature import local_binary_pattern, hog
from tqdm import tqdm 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

In [16]:
DATASET_PATH = './Dataset_Augmented/'
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 32

In [17]:
print('Getting file paths and labels')

image_paths = []
labels = []

positive_path = os.path.join(DATASET_PATH, 'Positive')
negative_path = os.path.join(DATASET_PATH, 'Negative')

for filename in os.listdir(positive_path):
    image_paths.append(os.path.join(positive_path, filename))
    labels.append(1)
    
for filename in os.listdir(negative_path):
    image_paths.append(os.path.join(negative_path, filename))
    labels.append(0)
    
image_paths = np.array(image_paths)
labels = np.array(labels)

X_train_paths, X_test_paths, y_train, y_test = train_test_split(
    image_paths, labels, test_size=0.25, random_state=42, stratify=labels
)

print(f'Training test size: {len(X_train_paths)}')
print(f'Testing test size: {len(X_test_paths)}')

Getting file paths and labels
Training test size: 1987
Testing test size: 663


# Getting the best descriptor and detector

## just detector: using lbp

In [18]:
def feature_generator_lbp(image_paths, labels, batch_size):
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i+batch_size]
            batch_labels = shuffled_labels[i:i+batch_size]
            
            batch_features = []
            
            for img_path in tqdm(batch_paths, desc='Batch Progress'):
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # gray_image_eq = cv2.equalizeHist(gray_image)
                
                lbp = local_binary_pattern(gray_image, P=8, R=1, method='uniform')
                
                (hist, _) = np.histogram(lbp.ravel(), bins = np.arange(0, 11), range=(0, 10))
                
                hist = hist.astype('float')
                
                hist /= (hist.sum() + 1e-6)
                
                batch_features.append(hist)
            
            yield np.array(batch_features), np.array(batch_labels)

In [19]:
# Verification
train_gen_lbp = feature_generator_lbp(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_batch_features, sample_batch_labels = next(train_gen_lbp) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_batch_features.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_batch_labels.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_batch_features[0]})') # 10 arrays

fetching one batch of feature vectors to test


Batch Progress: 100%|██████████| 32/32 [00:01<00:00, 17.97it/s]

pipeline complete, ready for training
shape of one batch of features: (32, 10)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [0.16161113 0.11670918 0.04316805 0.03103077 0.02742347 0.03350207
 0.04500159 0.11172672 0.17179528 0.25803173])


## detector with descriptor (fast + brief) - set up a baseline

In [20]:
def feature_generator_fast(image_paths, labels, batch_size):
    print("--- RUNNING THE NEW, CORRECTED FAST GENERATOR V2 ---")
    fast = cv2.FastFeatureDetector_create(nonmaxSuppression=False)
    fast.setThreshold(5)
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i + batch_size]
            batch_labels = shuffled_labels[i:i + batch_size]
            
            batch_features = []
            
            print(f'processing batch at index: {i}')
            for img_path in tqdm(batch_paths, desc='Batch Progress'):
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                keypoints = fast.detect(gray_image, None)
                
                keypoints, descriptors = brief.compute(gray_image, keypoints)
                
                if descriptors is not None:
                    feature_vector = np.mean(descriptors, axis=0)
                else:
                    feature_vector = np.zeros(32)
                
                batch_features.append(feature_vector)
            
            yield np.array(batch_features), np.array(batch_labels)

In [21]:
# Verification
train_gen_fast = feature_generator_fast(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_fast_batch_features, sample_fast_batch_labels = next(train_gen_fast) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_fast_batch_features.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_fast_batch_labels.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_fast_batch_features[0]})') # 10 arrays

fetching one batch of feature vectors to test
--- RUNNING THE NEW, CORRECTED FAST GENERATOR V2 ---
processing batch at index: 0


Batch Progress: 100%|██████████| 32/32 [00:01<00:00, 19.54it/s]

pipeline complete, ready for training
shape of one batch of features: (32, 32)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [ 78.55988024  96.00698603  70.41616766 162.17065868  88.98602794
  82.38223553 154.23752495 112.42215569  67.43013972 157.62774451
  95.01696607  95.93313373 165.50998004 172.01696607 107.03992016
  63.22355289 155.17764471 182.25948104 102.21457086 131.93612774
 130.01197605 100.05688623 162.99401198  82.20958084 162.39221557
 137.3752495  139.61377246 122.23852295 158.13972056  89.46307385
 158.45309381 164.91616766])


### fast is unreasonably fast with only 1.6s, since a detector + descriptor combo usually will take a while
so an isolated evaluation will be done

In [39]:
SAMPLE_SIZE = 100

fast = cv2.FastFeatureDetector_create(nonmaxSuppression=False)
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

def get_descriptor_count(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    keypoints_found = fast.detect(gray, None)
    keypoints_kept, descriptors = brief.compute(gray, keypoints_found)
    
    return len(descriptors) if descriptors is not None else 0

print("--- Analyzing Cracked Images ---")
positive_path = os.path.join(DATASET_PATH, 'Positive')
positive_files = [os.path.join(positive_path, fname) for fname in os.listdir(positive_path)[:SAMPLE_SIZE]]
positive_counts = [get_descriptor_count(path) for path in tqdm(positive_files, desc="Cracked")]

print("\n--- Analyzing Uncracked Images ---")
negative_path = os.path.join(DATASET_PATH, 'Negative')
negative_files = [os.path.join(negative_path, fname) for fname in os.listdir(negative_path)[:SAMPLE_SIZE]]
negative_counts = [get_descriptor_count(path) for path in tqdm(negative_files, desc="Uncracked")]

print("\n\n--- FINAL DIAGNOSTIC REPORT ---")
print(f"Average descriptors for CRACKED images: {np.mean(positive_counts):.2f}")
print(f"Average descriptors for UNCRACKED images: {np.mean(negative_counts):.2f}")
print(f"\nYour 'hero' image had {positive_counts[0]} descriptors.")

--- Analyzing Cracked Images ---


Cracked: 100%|██████████| 100/100 [00:05<00:00, 19.27it/s]



--- Analyzing Uncracked Images ---


Uncracked: 100%|██████████| 100/100 [00:05<00:00, 16.67it/s]



--- FINAL DIAGNOSTIC REPORT ---
Average descriptors for CRACKED images: 239.87
Average descriptors for UNCRACKED images: 55.82

Your 'hero' image had 99 descriptors.


--- FINAL DIAGNOSTIC REPORT ---

Average descriptors for CRACKED images: 239.87

Average descriptors for UNCRACKED images: 55.82

Your 'hero' image had 99 descriptors.

the result is pleasantly surprising with a discriminative score of ~4.3 (239 / 55). A nice baseline to have, but the detector was still picking up a significant amount of background texture noise, so to improve this, a higher threshold (30) will be trialed if it's true or not

In [40]:
SAMPLE_SIZE = 100

fast = cv2.FastFeatureDetector_create(threshold=30, nonmaxSuppression=False)
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

def get_descriptor_count(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    keypoints_found = fast.detect(gray, None)
    keypoints_kept, descriptors = brief.compute(gray, keypoints_found)
    
    return len(descriptors) if descriptors is not None else 0

print("--- Analyzing Cracked Images ---")
positive_path = os.path.join(DATASET_PATH, 'Positive')
positive_files = [os.path.join(positive_path, fname) for fname in os.listdir(positive_path)[:SAMPLE_SIZE]]
positive_counts = [get_descriptor_count(path) for path in tqdm(positive_files, desc="Cracked")]

print("\n--- Analyzing Uncracked Images ---")
negative_path = os.path.join(DATASET_PATH, 'Negative')
negative_files = [os.path.join(negative_path, fname) for fname in os.listdir(negative_path)[:SAMPLE_SIZE]]
negative_counts = [get_descriptor_count(path) for path in tqdm(negative_files, desc="Uncracked")]

print("\n\n--- FINAL DIAGNOSTIC REPORT ---")
print(f"Average descriptors for CRACKED images: {np.mean(positive_counts):.2f}")
print(f"Average descriptors for UNCRACKED images: {np.mean(negative_counts):.2f}")
print(f"\nYour 'hero' image had {positive_counts[0]} descriptors.")

--- Analyzing Cracked Images ---


Cracked: 100%|██████████| 100/100 [00:03<00:00, 25.99it/s]



--- Analyzing Uncracked Images ---


Uncracked: 100%|██████████| 100/100 [00:03<00:00, 25.75it/s]



--- FINAL DIAGNOSTIC REPORT ---
Average descriptors for CRACKED images: 49.96
Average descriptors for UNCRACKED images: 2.40

Your 'hero' image had 23 descriptors.


--- FINAL DIAGNOSTIC REPORT ---

Average descriptors for CRACKED images: 49.96

Average descriptors for UNCRACKED images: 2.40

Your 'hero' image had 23 descriptors.

As expected, a higher threshold will yield a better result. So for the comparison, FAST will use a threshold of 30 because it can distinguish cracks from paint texture. This calibration will be applied to both FAST and ORB (which also utilizes FAST in its algorithm) to ensure they were operating effectively on the dataset. Adaptive algorithms like AKAZE and MSER were evaluated at default settings as they inherently handle noise variation through their internal scale-space mechanisms inherently handle the local contrast variations without manual tuning

## orb

In [51]:
def feature_generator_orb(image_paths, labels, batch_size):
    orb = cv2.ORB_create(fastThreshold=30)
    
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i + batch_size]
            batch_labels = shuffled_labels[i:i + batch_size]
            
            batch_features = []
            
            for img_path in tqdm(batch_paths, desc='Batch Progress'):
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                keypoints, descriptors = orb.detectAndCompute(gray_image, None)
                
                if descriptors is not None:
                    feature_vector = np.mean(descriptors, axis=0)
                else:
                    feature_vector = np.zeros(32)
                
                batch_features.append(feature_vector)
            
            yield np.array(batch_features), np.array(batch_labels)

In [55]:
# Verification
train_gen_orb = feature_generator_orb(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_orb_batch_features, sample_orb_batch_labels = next(train_gen_orb) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_orb_batch_features.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_orb_batch_labels.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_orb_batch_features[0]})') # 10 arrays
print(f"Label of the zero-vector image: {sample_orb_batch_labels[0]}")

fetching one batch of feature vectors to test


Batch Progress: 100%|██████████| 32/32 [00:01<00:00, 24.44it/s]

pipeline complete, ready for training
shape of one batch of features: (32, 32)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [ 47.10714286 136.82142857 100.78571429 174.78571429  89.42857143
 115.57142857 141.42857143 147.03571429 103.53571429 103.71428571
 116.64285714 144.57142857 107.35714286 116.         146.60714286
 109.5        136.82142857 129.89285714  98.67857143 138.14285714
 160.92857143 142.46428571  58.46428571 191.78571429 112.96428571
 127.03571429 158.42857143 144.67857143  79.64285714 130.67857143
 135.32142857 116.85714286])
Label of the zero-vector image: 1


In [49]:
SAMPLE_SIZE = 1000  

def test_orb_params(fast_thresh=None, name="Default"):
    print(f"\n--- Testing ORB Configuration: {name} ---")
    
    if fast_thresh:
        orb = cv2.ORB_create(fastThreshold=fast_thresh)
    else:
        orb = cv2.ORB_create() 

    positive_path = os.path.join(DATASET_PATH, 'Positive')
    negative_path = os.path.join(DATASET_PATH, 'Negative')
    
    pos_files = [os.path.join(positive_path, f) for f in os.listdir(positive_path)[:SAMPLE_SIZE]]
    neg_files = [os.path.join(negative_path, f) for f in os.listdir(negative_path)[:SAMPLE_SIZE]]

    pos_counts = []
    for p in tqdm(pos_files, desc="Cracked"):
        img = cv2.imread(p); img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kps, desc = orb.detectAndCompute(gray, None)
        count = len(desc) if desc is not None else 0
        pos_counts.append(count)

    neg_counts = []
    for p in tqdm(neg_files, desc="Uncracked"):
        img = cv2.imread(p); img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kps, desc = orb.detectAndCompute(gray, None)
        count = len(desc) if desc is not None else 0
        neg_counts.append(count)

    avg_pos = np.mean(pos_counts)
    avg_neg = np.mean(neg_counts)
    ratio = avg_pos / (avg_neg + 1e-6)
    
    print(f"  -> Cracked Avg: {avg_pos:.2f}")
    print(f"  -> Uncracked Avg: {avg_neg:.2f}")
    print(f"  -> Ratio: {ratio:.2f}")

test_orb_params(None, 'Default')
test_orb_params(fast_thresh=30, name="Calibrated (Threshold 30)")


--- Testing ORB Configuration: Default ---


Uncracked: 100%|██████████| 1000/1000 [00:45<00:00, 21.87it/s]


  -> Cracked Avg: 139.38
  -> Uncracked Avg: 39.13
  -> Ratio: 3.56

--- Testing ORB Configuration: Calibrated (Threshold 30) ---


Uncracked: 100%|██████████| 1000/1000 [00:45<00:00, 21.87it/s]

  -> Cracked Avg: 87.53
  -> Uncracked Avg: 18.99
  -> Ratio: 4.61


```
--- Testing ORB Configuration: Default ---
-> Cracked Avg: 139.38
  -> Uncracked Avg: 39.13
  -> Ratio: 3.56

--- Testing ORB Configuration: Calibrated (Threshold 30) ---
-> Cracked Avg: 87.53
  -> Uncracked Avg: 18.99
  -> Ratio: 4.61
  
the improvement isn't as great as FAST most probably because ORB can handle a noise somewhat in it's inherent algorithm, but this is an improvement nonetheless so the new threshold will be used
```

## akaze

In [25]:
def feature_generator_akaze(image_paths, labels, batch_size):
    akaze = cv2.AKAZE_create()
    
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range (0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i + batch_size]
            batch_labels = shuffled_labels[i:i + batch_size]
            
            batch_features = []
            
            for img_path in tqdm(batch_paths, desc='Batch Progress'):
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                keypoints, descriptors = akaze.detectAndCompute(gray_img, None)
                
                if descriptors is not None:
                    feature_vectors = np.mean(descriptors, axis=0)
                else:
                    feature_vectors = np.zeros(61)
                
                batch_features.append(feature_vectors)
            
            yield np.array(batch_features), np.array(batch_labels)

In [26]:
# Verification
train_gen_akaze = feature_generator_akaze(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_akaze_batch_features, sample_akaze_batch_labels = next(train_gen_akaze) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_akaze_batch_features.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_akaze_batch_labels.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_akaze_batch_features[0]})') # 10 arrays

fetching one batch of feature vectors to test


Batch Progress: 100%|██████████| 32/32 [00:01<00:00, 19.49it/s]

pipeline complete, ready for training
shape of one batch of features: (32, 61)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [124.78947368  26.63157895 120.47368421  86.89473684 147.73684211
  14.42105263 132.         143.68421053 129.52631579  75.05263158
 157.57894737  52.21052632  43.73684211  84.         230.89473684
 104.57894737 181.68421053 169.26315789 137.52631579 226.05263158
 105.94736842 199.78947368 146.68421053  73.94736842  13.10526316
  35.89473684  39.94736842  37.68421053  63.68421053  13.89473684
 126.68421053  94.42105263 154.94736842 119.         107.63157895
 125.26315789 113.05263158 126.89473684  82.10526316  85.57894737
 137.63157895 156.47368421  84.57894737 117.68421053 160.
 109.05263158  34.68421053 148.42105263  38.52631579 198.78947368
  70.68421053  24.73684211 101.94736842 100.73684211  28.84210526
  20.73684211  35.78947368 221.63157895 212.31578947 242.36842105
  35.73684211])


## HOG

In [27]:
def feature_generator_hog(image_paths, labels, batch_size):
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i: i + batch_size]
            batch_labels = shuffled_labels[i: i + batch_size]
            
            batch_features = []
            
            for img_path in batch_paths:
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                hog_features = hog(
                    gray_image,
                    orientations=9,
                    pixels_per_cell=(8,8),
                    cells_per_block=(2,2),
                    transform_sqrt=True,
                    block_norm='L1'
                )
                
                batch_features.append(hog_features)
            
            yield np.array(batch_features), np.array(batch_labels)

In [28]:
# Verification
train_gen_hog = feature_generator_hog(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_hog_batch_features, sample_hog_batch_labels = next(train_gen_hog) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_hog_batch_features.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_hog_batch_labels.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_hog_batch_features[0]})') # 10 arrays

fetching one batch of feature vectors to test
pipeline complete, ready for training
shape of one batch of features: (32, 26244)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [0.03495404 0.02094397 0.02161804 ... 0.02134337 0.006198   0.02092129])


## MSER

In [29]:
def feature_generator_mser(image_paths, labels, batch_size):
    mser = cv2.MSER_create()
    
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i + batch_size]
            batch_labels = shuffled_labels[i:i + batch_size]
            
            batch_features = []
            
            for img_path in batch_paths:
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                regions, _ = mser.detectRegions(gray_image)
                
                total_area = 0
                
                if regions is not None:
                    total_area = sum(cv2.contourArea(region) for region in regions)
                
                batch_features.append([total_area])
            
            yield np.array(batch_features), np.array(batch_labels)
                    

In [30]:
# Verification
train_gen_mser = feature_generator_mser(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_mser_batch_features, sample_mser_batch_labels = next(train_gen_mser) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_mser_batch_features.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_mser_batch_labels.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_mser_batch_features[0]})') # 10 arrays

fetching one batch of feature vectors to test
pipeline complete, ready for training
shape of one batch of features: (32, 1)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [106654.5])


# pre test
a test for all the local descriptor pipelines first so that the pipelines can be evaluated first before doing a rigorous training using model evaluation

In [50]:
SAMPLE_SIZE = 10000

def get_descriptor_count_fast(image_path):
    fast = cv2.FastFeatureDetector_create(threshold=30, nonmaxSuppression=True)
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints = fast.detect(gray, None)
    _, descriptors = brief.compute(gray, keypoints)
    return len(descriptors) if descriptors is not None else 0

def get_descriptor_count_orb(image_path):
    orb = cv2.ORB_create(fastThreshold=30)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, descriptors = orb.detectAndCompute(gray, None)
    return len(descriptors) if descriptors is not None else 0

def get_descriptor_count_akaze(image_path):
    akaze = cv2.AKAZE_create()
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, descriptors = akaze.detectAndCompute(gray, None)
    return len(descriptors) if descriptors is not None else 0

def get_area_mser(image_path):
    mser = cv2.MSER_create()
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    regions, _ = mser.detectRegions(gray)
    return sum(cv2.contourArea(r) for r in regions) if regions is not None else 0

results = {}
diagnostics_to_run = {
    'FAST + BRIEF': get_descriptor_count_fast,
    'ORB': get_descriptor_count_orb,
    'AKAZE': get_descriptor_count_akaze,
    'MSER': get_area_mser
}

positive_files = [os.path.join(positive_path, fname) for fname in os.listdir(positive_path)[:SAMPLE_SIZE]]
negative_files = [os.path.join(negative_path, fname) for fname in os.listdir(negative_path)[:SAMPLE_SIZE]]

for name, helper_function in diagnostics_to_run.items():
    print(f'analyzing {name}')
    
    positive_counts = [helper_function(path) for path in tqdm(positive_files, desc=f'Cracked ({name})')]
    negative_counts = [helper_function(path) for path in tqdm(negative_files, desc=f'Uncracked ({name})')]
    
    results[name] = {
        'cracked_avg': np.mean(positive_counts),
        'uncracked_avg': np.mean(negative_counts)
    }

print('final pre-test diagnostic report')

for name, data in results.items():
    cracked_avg = data['cracked_avg']
    uncracked_avg = data['uncracked_avg']
    
    ratio = cracked_avg / (uncracked_avg + 1e-6)
    
    print(f'algorithm: {name}')
    print(f'Avg features on CRACKED images: {cracked_avg}')
    print(f'Avg features on UNCRACKED images: {uncracked_avg}')
    print(f'Discriminative ratio (Cracked/Uncracked): {ratio}')

analyzing FAST + BRIEF


Uncracked (FAST + BRIEF): 100%|██████████| 1325/1325 [01:46<00:00, 12.44it/s]


analyzing ORB


Uncracked (ORB): 100%|██████████| 1325/1325 [01:19<00:00, 16.69it/s]


analyzing AKAZE


Uncracked (AKAZE): 100%|██████████| 1325/1325 [01:20<00:00, 16.55it/s]


analyzing MSER


Uncracked (MSER): 100%|██████████| 1325/1325 [01:22<00:00, 16.02it/s]

final pre-test diagnostic report
algorithm: FAST + BRIEF
Avg features on CRACKED images: 4465.603773584906
Avg features on UNCRACKED images: 8068.931320754717
Discriminative ratio (Cracked/Uncracked): 0.5534318728857132
algorithm: ORB
Avg features on CRACKED images: 78.40754716981132
Avg features on UNCRACKED images: 31.244528301886792
Discriminative ratio (Cracked/Uncracked): 2.509480825018428
algorithm: AKAZE
Avg features on CRACKED images: 4.794716981132075
Avg features on UNCRACKED images: 1.9486792452830188
Discriminative ratio (Cracked/Uncracked): 2.4604944770893953
algorithm: MSER
Avg features on CRACKED images: 32002.570188679245
Avg features on UNCRACKED images: 23418.564150943395
Discriminative ratio (Cracked/Uncracked): 1.3665470684300474


```
final pre-test diagnostic report
algorithm: FAST + BRIEF
Avg features on CRACKED images: 4465.603773584906
Avg features on UNCRACKED images: 8068.931320754717
Discriminative ratio (Cracked/Uncracked): 0.5534318728857132
algorithm: ORB
Avg features on CRACKED images: 78.40754716981132
Avg features on UNCRACKED images: 31.244528301886792
Discriminative ratio (Cracked/Uncracked): 2.509480825018428
algorithm: AKAZE
Avg features on CRACKED images: 4.794716981132075
Avg features on UNCRACKED images: 1.9486792452830188
Discriminative ratio (Cracked/Uncracked): 2.4604944770893953
algorithm: MSER
Avg features on CRACKED images: 32002.570188679245
Avg features on UNCRACKED images: 23418.564150943395
Discriminative ratio (Cracked/Uncracked): 1.3665470684300474
```

## result:
FAST and MSER will be excluded from the final evaluation. The final score test will be a competition between our top candidates: AKAZE, ORB, LBP, and HOG

### Why FAST and MSER are excluded?
Both of these algorithms has a very bad ratio ~1 or even less, meaning that for every one flag of signal, there is also 1 or more noise. Meaning it can't distinguish a crack from a shadow
  
### Hypothesis: a Global Descriptor will be better than a Local one
The relatively low results of all the keypoint detectors (< 3.0) suggests the 'local descriptor hypothesis': that cracks are best defined by specific high-contrast points, is weak for this specific dataset due to the high texture noise in here

Consequently, the scope of the final evaluation has been expanded to prioritize 'Global Descriptors' like LBP (Global Texture) and HOG (Global Shape). The updated hypothesis is that analyzing the overall texture or edge distribution of the entire image will be more robust against local paint irregularities than trying to isolate the individual keypoints

# Feature Selection

In [57]:
NUM_TRAINING_STEPS = len(X_train_paths) // BATCH_SIZE

pipeline_generators = {
    'LBP': feature_generator_lbp,
    'HOG': feature_generator_hog,
    'AKAZE': feature_generator_akaze,
    'ORB': feature_generator_orb 
}

def extract_test_features(extractor_name, paths):
    print(f'extracting test features for {extractor_name}')
    
    test_features = []
    
    akaze = cv2.AKAZE_create()
    orb = cv2.ORB_create(fastThreshold=30)
    
    for img_path in tqdm(paths, desc=f'Testing {extractor_name}'):
        image = cv2.imread(img_path)
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if extractor_name == 'LBP':
            lbp = local_binary_pattern(gray_image, P=8, R=1, method='uniform')
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
            hist = hist.astype('float')
            hist /= (hist.sum() + 1e-6)
            test_features.append(hist)
        elif extractor_name == 'HOG':
            hog_features = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), transform_sqrt=True, block_norm='L1')
            test_features.append(hog_features)
        elif extractor_name == 'AKAZE':
            _, descriptors = akaze.detectAndCompute(gray_image, None)
            if descriptors is not None: 
                feature_vector = np.mean(descriptors, axis=0)
            else:
                feature_vector = np.zeros(61)
            test_features.append(feature_vector)
        elif extractor_name == 'ORB':
            _, descriptors = orb.detectAndCompute(gray_image, None)
            feature_vector = np.mean(descriptors, axis=0) if descriptors is not None else np.zeros(32)
            test_features.append(feature_vector)
        
    return np.array(test_features)

results = {}

for pipeline_name, generator_function in pipeline_generators.items():
    print(f'training baseline model for: {pipeline_name}')
    
    model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
    train_generator = generator_function(X_train_paths, y_train, BATCH_SIZE)
    
    for _ in tqdm(range(int(NUM_TRAINING_STEPS)), desc=f'Training {pipeline_name}'):
        X_batch, y_batch = next(train_generator)
        model.partial_fit(X_batch, y_batch, classes=np.array([0, 1]))
    
    X_test_features = extract_test_features(pipeline_name, X_test_paths)
    y_pred = model.predict(X_test_features)
    report = classification_report(y_test, y_pred, target_names=['No Crack (0)', 'Crack (1)'], output_dict=True)
    results[pipeline_name] = report

print('final baseline comparison report')

best_pipeline = max(results, key=lambda p: results[p]['macro avg']['f1-score'])

for pipeline_name, report in results.items():
    f1_crack = report['Crack (1)']['f1-score']
    print("==========================================")
    print(f"               {pipeline_name} {'WINNER' if pipeline_name == best_pipeline else ''}")
    print("==========================================")
    print(f"              precision    recall  f1-score   support")
    print(f"No Crack (0)      {report['No Crack (0)']['precision']:.2f}         {report['No Crack (0)']['recall']:.2f}      {report['No Crack (0)']['f1-score']:.2f}      {report['No Crack (0)']['support']}")
    print(f"   Crack (1)      {report['Crack (1)']['precision']:.2f}         {report['Crack (1)']['recall']:.2f}      {report['Crack (1)']['f1-score']:.2f}      {report['Crack (1)']['support']}")
    print(f"\n   Accuracy                           {report['accuracy']:.2f}     {report['macro avg']['support']}")
    print(f"   Macro Avg      {report['macro avg']['precision']:.2f}         {report['macro avg']['recall']:.2f}      {report['macro avg']['f1-score']:.2f}      {report['macro avg']['support']}")
    print(f"Weighted Avg      {report['weighted avg']['precision']:.2f}         {report['weighted avg']['recall']:.2f}      {report['weighted avg']['f1-score']:.2f}      {report['weighted avg']['support']}")

training baseline model for: LBP


Training LBP: 100%|██████████| 62/62 [01:52<00:00,  1.81s/it]


extracting test features for LBP


Testing LBP: 100%|██████████| 663/663 [00:37<00:00, 17.86it/s]


training baseline model for: HOG


...

training baseline model for: ORB


Training ORB: 100%|██████████| 62/62 [01:22<00:00,  1.33s/it]


extracting test features for ORB


Testing ORB: 100%|██████████| 663/663 [00:27<00:00, 23.73it/s]

final baseline comparison report
               LBP 
              precision    recall  f1-score   support
No Crack (0)      0.53         0.94      0.68      332.0
   Crack (1)      0.74         0.16      0.26      331.0

   Accuracy                           0.55     663.0
   Macro Avg      0.63         0.55      0.47      663.0
Weighted Avg      0.63         0.55      0.47      663.0
               HOG WINNER
              precision    recall  f1-score   support
No Crack (0)      0.73         0.58      0.65      332.0
   Crack (1)      0.65         0.78      0.71      331.0

   Accuracy                           0.68     663.0
   Macro Avg      0.69         0.68      0.68      663.0
Weighted Avg      0.69         0.68      0.68      663.0
               AKAZE 
              precision    recall  f1-score   support
No Crack (0)      0.55         0.96      0.70      332.0
   Crack (1)      0.84         0.21      0.34      331.0

   Accuracy                           0.59     663.0
   Ma

In [33]:
import json

In [58]:
try:
    with open('./Result/baseline_results.json', 'r') as f:
        loaded_results = json.load(f)

    lbp_f1_score = loaded_results['LBP']['Crack (1)']['f1-score']
    print(f"The LBP F1-score for cracks was: {lbp_f1_score}")
except:
    results_filename = './Result/baseline_results.json'
    print(f'saving to {results_filename}')

    with open(results_filename, 'w') as f:
        json.dump(results, f, indent=4)

    print('saved successfully')

saving to ./Result/baseline_results.json
saved successfully
